In [1]:
import cProfile
import pstats
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np

from superfv import AdvectionSolver
import superfv.initial_conditions as ic
import superfv.visualization as vis

In [2]:
def passive_square(x, y, z):
    xc, yc = x - 0.5, y - 0.75
    return np.where(
        (np.abs(xc) < 0.15) & (np.abs(yc) < 0.15),
        1.0,
        0.0,
    )


sim = AdvectionSolver(
    ic=lambda idx, x, y, z: ic.slotted_disk(idx, x, y, z, rotation="ccw"),
    ic_passives={"passive_square": passive_square},
    bcx="ic",
    bcy="ic",
    nx=64,
    ny=64,
    nz=1,
    p=3,
    ZS=True,
    PAD={"rho": (0, 1)},
    interpolation_scheme="gauss-legendre",
)

# sim.run([np.pi, 2 * np.pi])

In [3]:
pr = cProfile.Profile()
pr.enable()
sim.run(n=20)
pr.disable()
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumtime")
ps.print_stats()
print(s.getvalue())

100%|██████████| 20/20 [00:01<00:00, 10.73it/s]


         188800 function calls (185737 primitive calls) in 1.929 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      3/2    0.000    0.000    1.928    0.964 /Users/jonathan/miniconda3/envs/superfv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3543(run_code)
     11/2    0.000    0.000    1.928    0.964 {built-in method builtins.exec}
   2194/1    0.018    0.000    1.875    1.875 /Users/jonathan/Desktop/superfv/superfv/tools/timer.py:8(wrapper)
      2/1    0.000    0.000    1.874    1.874 /Users/jonathan/Desktop/superfv/superfv/finite_volume_solver.py:1437(run)
        1    0.000    0.000    1.874    1.874 /Users/jonathan/Desktop/superfv/superfv/explicit_ODE_solver.py:455(rk4)
        1    0.000    0.000    1.874    1.874 /Users/jonathan/Desktop/superfv/superfv/explicit_ODE_solver.py:216(integrate)
       20    0.003    0.000    1.846    0.092 /Users/jonathan/Desktop/superfv/superfv/explicit_ODE_solver.p

In [4]:
np.min(sim.minisnapshots["min_rho"]), np.max(sim.minisnapshots["max_rho"])

(np.float64(-2.0736719817022717e-18), np.float64(1.0))

In [5]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
vis.plot_2d_slice(sim, ax, "rho", x=None, y=None)
# add colorbar
ax.figure.colorbar(ax.images[0], ax=ax)

In [6]:
sim.timer.print_report()

Category                                              Calls      Cumulative Time
----------------------------------------------------------------------------------
!ExplicitODESolver.integrate.body                         1                 1.89
!FiniteVolumeSolver.run                                   1                 1.93
AdvectionSolver.advection_upwind                        160                 0.06
AdvectionSolver.compute_dt                               20                 0.00
AdvectionSolver.minisnapshot                             41                 0.00
ExplicitODESolver.integrate                               1                 1.93
ExplicitODESolver.minisnapshot                           41                 0.00
ExplicitODESolver.take_step                              20                 1.85
FiniteVolumeSolver.adaptive_dt_criterion                 20                 0.00
FiniteVolumeSolver.apply_bc                             402                 0.32
FiniteVolumeSolver.compute